In [1]:
import argparse
import datetime
import os,json
import pprint

import numpy as np
import torch, random
from torch import nn
from torch.utils.tensorboard import SummaryWriter

import gymnasium as gym

from tianshou.data import Collector, ReplayBuffer, VectorReplayBuffer
from tianshou.policy import RainbowPolicy, TD3Policy, DQNPolicy
from tianshou.trainer import offpolicy_trainer
from tianshou.exploration import GaussianNoise
from tianshou.utils import TensorboardLogger
from tianshou.utils.net.common import Net
from tianshou.utils.net.continuous import Actor, Critic
from tianshou.env import ShmemVectorEnv

In [2]:
seed = 2023
torch.manual_seed(seed)
np.random.seed(seed)

In [3]:
def make_env(seed, task, training_num, test_num):
    env = gym.make(task)
    train_envs = ShmemVectorEnv(
        [lambda: gym.make(task) for _ in range(training_num)]
    )
    test_envs = ShmemVectorEnv([lambda: gym.make(task) for _ in range(test_num)])
    # env.seed(seed)
    train_envs.seed(seed)
    test_envs.seed(seed)
    
    return env, train_envs, test_envs

In [4]:
env, train_envs, test_envs = make_env(seed = 2023, task = "ALE/Breakout-v5", training_num = 1, test_num = 1)

In [5]:
class Net(nn.Module):
    def __init__(self, n_channels, filters, strides, obs_space, action_space):
        super().__init__()
        
        in_channel = obs_space.shape[-1]
        self.cnn = nn.Sequential(
            nn.Conv2d(in_channel, n_channels[0], filters[0], strides[0]),
            nn.Conv2d(n_channels[0], n_channels[1], filters[1], strides[1]),
            nn.Conv2d(n_channels[1], n_channels[2], filters[2], strides[2]),
        )
        
        with torch.no_grad():
            obs_sample = torch.tensor(obs_space.sample()).float()
            obs_sample = torch.permute(obs_sample, (2, 0, 1))
            output_dim = self.cnn(obs_sample).flatten().size(dim = 0)
            
        self.linear = nn.Sequential(
            # nn.Linear(output_dim, int(np.prod(action_space.shape))),
            nn.Linear(output_dim, action_space.n),
        )

    def forward(self, obs, state=None, info={}):
        if not isinstance(obs, torch.Tensor):
            obs = torch.tensor(obs, dtype=torch.float)
        batch = obs.shape[0]
        
        obs = torch.permute(obs, (0, 3, 1, 2))
        preprocessed = self.cnn(obs).flatten(1)
        logits = self.linear(preprocessed.view(batch, -1)).unsqueeze(dim = -1)

        return logits, state

In [6]:
device = 'cpu'
n_channels = [32, 64, 64]
strides = [4, 2, 1]
filters = [8, 4, 1]
lr = 1e-4
gamma = 0.99
n_step = 1
start_timesteps = 25000
#Collector
buffer_size = 100000

#Trainer
max_epoch = 200
step_per_epoch = 1000
step_per_collect = 1
test_num = 10
batch_size = 32
update_per_step = 1


In [7]:
# model
model = Net(n_channels, filters, strides, env.observation_space, env.action_space)
model_optim = torch.optim.Adam(model.parameters(), lr=lr)

policy = RainbowPolicy(
    model,
    model_optim,
    discount_factor=gamma,
    estimation_step=n_step,
)


In [8]:
# collector
buffer = ReplayBuffer(buffer_size)

train_collector = Collector(policy, train_envs, buffer, exploration_noise=True)
test_collector = Collector(policy, test_envs, exploration_noise = True)
train_collector.collect(n_step = start_timesteps, random=True)

{'n/ep': 133,
 'n/st': 25000,
 'rews': array([2., 4., 0., 0., 1., 1., 1., 1., 0., 1., 0., 0., 2., 1., 2., 0., 1.,
        0., 0., 2., 1., 2., 3., 3., 1., 3., 2., 0., 0., 2., 3., 2., 3., 2.,
        1., 1., 0., 3., 1., 0., 1., 2., 2., 0., 3., 0., 0., 1., 0., 2., 0.,
        1., 1., 3., 0., 2., 0., 2., 0., 4., 0., 1., 2., 0., 1., 0., 0., 0.,
        0., 2., 5., 3., 0., 2., 3., 2., 0., 1., 2., 0., 2., 0., 4., 2., 1.,
        0., 0., 1., 1., 3., 2., 2., 1., 2., 1., 1., 3., 1., 3., 1., 2., 2.,
        0., 3., 2., 1., 1., 0., 0., 0., 0., 3., 1., 0., 0., 5., 1., 0., 1.,
        2., 4., 2., 1., 3., 0., 0., 0., 0., 2., 0., 1., 3., 1.]),
 'lens': array([209, 279, 133, 128, 193, 176, 177, 178, 154, 176, 142, 139, 197,
        189, 207, 135, 168, 134, 137, 190, 180, 203, 263, 239, 173, 247,
        217, 138, 130, 260, 273, 211, 253, 255, 172, 156, 125, 240, 165,
        132, 163, 201, 212, 128, 231, 138, 128, 183, 133, 212, 162, 179,
        189, 263, 133, 205, 135, 227, 136, 291, 136, 170, 212, 1

In [9]:
# log
now = datetime.datetime.now().strftime("%y%m%d-%H%M%S")
algo_name = "rainbow_baselines"
log_name = os.path.join("breakout", algo_name, str(seed), now)
log_path = os.path.join("log_results", log_name)

# logger
writer = SummaryWriter(log_path)
# writer.add_text("args", str(args))
logger = TensorboardLogger(writer)

def save_best_fn(policy):
    torch.save(policy.state_dict(), os.path.join(log_path, "policy.pth"))

In [10]:
result = offpolicy_trainer(
    policy,
    train_collector,
    test_collector,
    max_epoch,
    step_per_epoch,
    step_per_collect,
    test_num,
    batch_size,
    save_best_fn=save_best_fn,
    logger=logger,
    train_fn=lambda epoch, env_step: policy.set_eps(0.2),
    test_fn=lambda epoch, env_step: policy.set_eps(0.1),
    update_per_step=update_per_step,
    test_in_train = False
)
pprint.pprint(result)

Epoch #1: 1001it [00:51, 19.35it/s, env_step=1000, len=282, loss=-14994881957068.801, n/ep=0, n/st=1, rew=3.00]        


Epoch #1: test_reward: 1.800000 ± 1.249000, best_reward: 2.700000 ± 2.368544 in #0


Epoch #2: 1001it [00:59, 16.73it/s, env_step=2000, len=133, loss=-237662449350737.906, n/ep=0, n/st=1, rew=0.00]       


Epoch #2: test_reward: 1.900000 ± 1.640122, best_reward: 2.700000 ± 2.368544 in #0


Epoch #3: 1001it [01:03, 15.80it/s, env_step=3000, len=162, loss=-1029486274286714.875, n/ep=0, n/st=1, rew=1.00]      


...

Epoch #198: test_reward: 1.800000 ± 1.249000, best_reward: 2.900000 ± 1.640122 in #161


Epoch #199: 1001it [00:51, 19.57it/s, env_step=199000, len=237, loss=-2628661742635301994496.000, n/ep=0, n/st=1, rew=3.00]


Epoch #199: test_reward: 1.700000 ± 1.268858, best_reward: 2.900000 ± 1.640122 in #161


Epoch #200: 1001it [00:51, 19.43it/s, env_step=200000, len=231, loss=-2680093804580044275712.000, n/ep=0, n/st=1, rew=2.00]


Epoch #200: test_reward: 1.400000 ± 1.113553, best_reward: 2.900000 ± 1.640122 in #161
{'best_result': '2.90 ± 1.64',
 'best_reward': 2.9,
 'duration': '11944.42s',
 'test_episode': 2010,
 'test_speed': '301.54 step/s',
 'test_step': 451988,
 'test_time': '1498.91s',
 'train_episode': 1056,
 'train_speed': '19.15 step/s',
 'train_step': 200000,
 'train_time/collector': '938.47s',
 'train_time/model': '9507.03s'}


In [11]:
support = torch.nn.Parameter(
            torch.linspace(-10, 10, 51),
            requires_grad=False,
        )
support

Parameter containing:
tensor([-10.0000,  -9.6000,  -9.2000,  -8.8000,  -8.4000,  -8.0000,  -7.6000,
         -7.2000,  -6.8000,  -6.4000,  -6.0000,  -5.6000,  -5.2000,  -4.8000,
         -4.4000,  -4.0000,  -3.6000,  -3.2000,  -2.8000,  -2.4000,  -2.0000,
         -1.6000,  -1.2000,  -0.8000,  -0.4000,   0.0000,   0.4000,   0.8000,
          1.2000,   1.6000,   2.0000,   2.4000,   2.8000,   3.2000,   3.6000,
          4.0000,   4.4000,   4.8000,   5.2000,   5.6000,   6.0000,   6.4000,
          6.8000,   7.2000,   7.6000,   8.0000,   8.4000,   8.8000,   9.2000,
          9.6000,  10.0000])

In [16]:
support * torch.tensor([[2], [3]])

tensor([[-20.0000, -19.2000, -18.4000, -17.6000, -16.8000, -16.0000, -15.2000,
         -14.4000, -13.6000, -12.8000, -12.0000, -11.2000, -10.4000,  -9.6000,
          -8.8000,  -8.0000,  -7.2000,  -6.4000,  -5.6000,  -4.8000,  -4.0000,
          -3.2000,  -2.4000,  -1.6000,  -0.8000,   0.0000,   0.8000,   1.6000,
           2.4000,   3.2000,   4.0000,   4.8000,   5.6000,   6.4000,   7.2000,
           8.0000,   8.8000,   9.6000,  10.4000,  11.2000,  12.0000,  12.8000,
          13.6000,  14.4000,  15.2000,  16.0000,  16.8000,  17.6000,  18.4000,
          19.2000,  20.0000],
        [-30.0000, -28.8000, -27.6000, -26.4000, -25.2000, -24.0000, -22.8000,
         -21.6000, -20.4000, -19.2000, -18.0000, -16.8000, -15.6000, -14.4000,
         -13.2000, -12.0000, -10.8000,  -9.6000,  -8.4000,  -7.2000,  -6.0000,
          -4.8000,  -3.6000,  -2.4000,  -1.2000,   0.0000,   1.2000,   2.4000,
           3.6000,   4.8000,   6.0000,   7.2000,   8.4000,   9.6000,  10.8000,
          12.0000,  13

In [12]:
a = torch.tensor([1,2,3])
a

tensor([1, 2, 3])

In [14]:
a.unsqueeze(dim = -1)

tensor([[1],
        [2],
        [3]])